# Australia Domestic Flights

We use a database provided by the **Bureau of Infrastructure and Transport Research Economics** downloaded from https://data.gov.au/dataset/ds-dga-c5029f2a-39b3-4aef-8ae1-73e7962f6170/details

The dataset contain 11 instances:

+ City1: A city each aircraft takes-off from
+ City2: A city each aircraft lands on.
+ Month: (I couldn’t find any definitions)
+ Passenger_Trips: (I couldn’t find any definitions)
+ Aircraft_Trips: The number of flight stages. A return gflight counts as two aircraft trips.
+ Passenger_Load_Factor: The total revenue passenger kilometres performed as a percentage of the total available seat kilometres.
+ Distance_GC_(km): Great Circle Distances between 2 airports
+ RPKs: Revenue Passenger Kilometres. calculated by multiplying the number of revenue passengers travelling on each flight stage, by the distance in kilometres between the ports. the distances used are great circle kilometres.
+ ASKs: It stands for “Available Seat Kilometres”. It is calculates by multiplying the number of seats availble on each flight stage, by the distance in kilometres between the ports. The distance used are Great Circle Distances.
+ Seats: Seats available in each route in that month
+ “Flight Statge” means the operation of an aircraft from take-off to landing.

*We will answer the following questions:*

+ Most popular routes
+ Which city shows a greater growth in flights **land on**?
+ Which was the best year for the australian domestic flights?
+ New cities added in the routes.

*Potential questions.*

+ Cities that have more growth potential.
+ **Try any algorithm of time series to predict the development of the flights**


In [40]:
import scipy
from scipy import stats

#librerías necesarias:

# Data handling
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Visualization
import seaborn as sns
#import plotly.express as px
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation

%matplotlib inline 
pd.set_option("max_rows", None)

## Dataset

In [90]:
data = pd.read_csv('https://data.gov.au/data/dataset/c5029f2a-39b3-4aef-8ae1-73e7962f6170/resource/677d307f-6a1f-4de4-9b85-5e1aa7074423/download/dom_citypairs_web.csv').dropna()

In [91]:
data.head()

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num
0,ADELAIDE,ALICE SPRINGS,30682,15743,143,81.8,1316,20717788.0,25327369.0,19246,1984,1
1,ADELAIDE,BRISBANE,30682,3781,32,89.8,1622,6132782.0,6829379.0,4210,1984,1
2,ADELAIDE,CANBERRA,30682,1339,12,94.7,972,1301508.0,1374348.0,1414,1984,1
3,ADELAIDE,DARWIN,30682,3050,33,66.8,2619,7987950.0,11958009.0,4566,1984,1
4,ADELAIDE,GOLD COAST,30682,1596,16,88.5,1607,2564772.0,2898047.0,1803,1984,1


In [92]:
data.shape

(24157, 12)

In [93]:
data.dtypes

City1                     object
City2                     object
Month                      int64
Passenger_Trips            int64
Aircraft_Trips             int64
Passenger_Load_Factor     object
Distance_GC_(km)           int64
RPKs                     float64
ASKs                     float64
Seats                      int64
Year                       int64
Month_num                  int64
dtype: object

### Period Stamp

In [94]:
data['Date'] = data['Month_num'].map(str) + '/' + data['Year'].map(str) 

In [95]:
data.head()

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num,Date
0,ADELAIDE,ALICE SPRINGS,30682,15743,143,81.8,1316,20717788.0,25327369.0,19246,1984,1,1/1984
1,ADELAIDE,BRISBANE,30682,3781,32,89.8,1622,6132782.0,6829379.0,4210,1984,1,1/1984
2,ADELAIDE,CANBERRA,30682,1339,12,94.7,972,1301508.0,1374348.0,1414,1984,1,1/1984
3,ADELAIDE,DARWIN,30682,3050,33,66.8,2619,7987950.0,11958009.0,4566,1984,1,1/1984
4,ADELAIDE,GOLD COAST,30682,1596,16,88.5,1607,2564772.0,2898047.0,1803,1984,1,1/1984


In [96]:
data_1 = pd.to_datetime(data['Date'])
data['Date'] = pd.PeriodIndex(data_1, freq='M')
data[:5]

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num,Date
0,ADELAIDE,ALICE SPRINGS,30682,15743,143,81.8,1316,20717788.0,25327369.0,19246,1984,1,1984-01
1,ADELAIDE,BRISBANE,30682,3781,32,89.8,1622,6132782.0,6829379.0,4210,1984,1,1984-01
2,ADELAIDE,CANBERRA,30682,1339,12,94.7,972,1301508.0,1374348.0,1414,1984,1,1984-01
3,ADELAIDE,DARWIN,30682,3050,33,66.8,2619,7987950.0,11958009.0,4566,1984,1,1984-01
4,ADELAIDE,GOLD COAST,30682,1596,16,88.5,1607,2564772.0,2898047.0,1803,1984,1,1984-01


### Routes

In [97]:
data['Routes'] = data['City1'] + ' - ' + data['City2']

In [98]:
# Check if there are duplicated routes (ex. ADELAIDE-SYDNEY  SYDNEY-ADELAIDE)

data_2 = data[data['City1'] == 'SYDNEY']

In [99]:
data_2

,City1,City2,Month,Passenger_Trips,Aircraft_Trips,Passenger_Load_Factor,Distance_GC_(km),RPKs,ASKs,Seats,Year,Month_num,Date,Routes
41,SYDNEY,TOWNSVILLE,30682,4262,50,88.4,1690,7202780.0,8147941.0,4821,1984,1,1984-01,SYDNEY - TOWNSVILLE
42,SYDNEY,WAGGA WAGGA,30682,3843,139,56.5,367,1410381.0,2496250.0,6802,1984,1,1984-01,SYDNEY - WAGGA WAGGA
82,SYDNEY,TOWNSVILLE,30713,1306,17,72.2,1690,2207140.0,3056981.0,1809,1984,2,1984-02,SYDNEY - TOWNSVILLE
83,SYDNEY,WAGGA WAGGA,30713,4295,149,56,367,1576265.0,2814759.0,7670,1984,2,1984-02,SYDNEY - WAGGA WAGGA
124,SYDNEY,TOWNSVILLE,30742,1041,13,81.6,1690,1759290.0,2155993.0,1276,1984,3,1984-03,SYDNEY - TOWNSVILLE
125,SYDNEY,WAGGA WAGGA,30742,4732,148,61.5,367,1736644.0,2823811.0,7694,1984,3,1984-03,SYDNEY - WAGGA WAGGA
167,SYDNEY,TOWNSVILLE,30773,1680,18,95.1,1690,2839200.0,2985489.0,1767,1984,4,1984-04,SYDNEY - TOWNSVILLE
168,SYDNEY,WAGGA WAGGA,30773,5035,143,67.7,367,1847845.0,2729461.0,7437,1984,4,1984-04,SYDNEY - WAGGA WAGGA
209,SYDNEY,TOWNSVILLE,30803,6126,76,76,1690,10352940.0,13622289.0,8061,1984,5,1984-05,SYDNEY - TOWNSVILLE
210,SYDNEY,WAGGA WAGGA,30803,5092,149,65.7,367,1868764.0,2844390.0,7750,1984,5,1984-05,SYDNEY - WAGGA WAGGA


When we filter only by Sydney we get only 2 cities from Sydney (Wagga-Wagga and Townsville). That ensures our assumption that we only have the one way route in the dataset.

## Most Popular Routes

In this section we are going to explore the most popular domestic routes and how they change during the years.

### Routes along the years

Seats available in each route during the years,

In [100]:
routes = pd.DataFrame(data, columns=['Routes', 'Seats', 'Year'])

In [101]:
routes_1 = pd.pivot_table(routes, index= "Routes", columns = "Year", aggfunc= "sum")
routes_1

Seats                                   \
Year                              1984       1985       1986       1987   
Routes                                                                    
ADELAIDE - ALICE SPRINGS      198702.0   210157.0   221362.0   217725.0   
ADELAIDE - BRISBANE            53500.0    54287.0    62707.0    85384.0   
ADELAIDE - CANBERRA            17961.0    35662.0    41591.0    35422.0   
ADELAIDE - DARWIN              40422.0    39171.0    30722.0    13643.0   
ADELAIDE - GOLD COAST          21074.0    22610.0    20808.0    20619.0   
ADELAIDE - MELBOURNE          935202.0  1023768.0  1055457.0  1032130.0   
ADELAIDE - PERTH              236174.0   249222.0   275180.0   300925.0   
ADELAIDE - PORT LINCOLN            NaN        NaN        NaN        NaN   
ADELAIDE - SYDNEY             544027.0   576891.0   594774.0   605772.0   
ALBURY - SYDNEY               152490.0   186380.0   209765.0   198023.0   
ALICE SPRINGS - DARWIN        130926.0   167801.0   230223.0   259629.0   
ALICE SPRINGS - MELBOURNE          NaN        NaN        NaN        NaN   
ALICE SPRINGS - SYDNEY         43154.0    70319.0    94376.0   127641.0   
ARMIDALE - SYDNEY                  NaN        NaN        NaN        NaN   
AYERS ROCK - SYDNEY                NaN        NaN        NaN        NaN   
BALLINA - SYDNEY                   NaN        NaN        NaN        NaN   
BRISBANE - CAIRNS              31202.0    35803.0    74028.0   162403.0   
BRISBANE - CANBERRA             8903.0    13151.0     7451.0     4886.0   
BRISBANE - DARWIN              25377.0    47583.0    46888.0    32808.0   
BRISBANE - EMERALD                 NaN        NaN        NaN        NaN   
BRISBANE - GLADSTONE               NaN        NaN        NaN        NaN   
BRISBANE - HAMILTON ISLAND         NaN        NaN        NaN        NaN   
BRISBANE - HOBART                  NaN        NaN        NaN        NaN   
BRISBANE - MACKAY              11134.0    18966.0    38391.0    28245.0   
BRISBANE - MELBOURNE          441402.0   494479.0   501461.0   555477.0   
BRISBANE - MORANBAH                NaN        NaN        NaN        NaN   
BRISBANE - MOUNT ISA               NaN        NaN        NaN        NaN   
BRISBANE - NEWCASTLE               NaN        NaN        NaN        NaN   
BRISBANE - PERTH                7976.0     7756.0     4416.0     6238.0   
BRISBANE - PROSERPINE         108123.0    96910.0    86998.0    77289.0   
BRISBANE - ROCKHAMPTON        331593.0   340905.0   356845.0   380831.0   
BRISBANE - SYDNEY            1605668.0  1662018.0  1753943.0  1884587.0   
BRISBANE - TOWNSVILLE         498713.0   503104.0   517154.0   538824.0   
BROOME - PERTH                  7935.0     8934.0    12843.0    10614.0   
BUNDABERG - BRISBANE               NaN        NaN        NaN        NaN   
CAIRNS - MELBOURNE                 NaN      211.0     2113.0    16027.0   
CAIRNS - SYDNEY                 1994.0     5077.0    38814.0    61141.0   
CAIRNS - TOWNSVILLE                NaN        NaN        NaN        NaN   
CANBERRA - MELBOURNE          546249.0   586127.0   628185.0   668279.0   
CANBERRA - SYDNEY             809558.0   922608.0   974020.0   968795.0   
COFFS HARBOUR - SYDNEY         94456.0   106769.0   102895.0   110572.0   
DARWIN - MELBOURNE                 NaN        NaN        NaN        NaN   
DARWIN - PERTH                     NaN        NaN        NaN        NaN   
DARWIN - SYDNEY                 3566.0     3624.0    16019.0    15250.0   
DEVONPORT - MELBOURNE         134157.0   164059.0   209494.0   195768.0   
DUBBO - SYDNEY                 91571.0    97477.0    96712.0    87183.0   
GERALDTON - PERTH                  NaN        NaN        NaN        NaN   
GOLD COAST - SYDNEY           538586.0   603066.0   687232.0   806782.0   
HAMILTON ISLAND - MELBOURNE        NaN        NaN        NaN        NaN   
HAMILTON ISLAND - SYDNEY           NaN        NaN        NaN        NaN   
HERVEY BAY - SYDNEY                NaN        NaN        N

por que me aparece NaN los seats de Adelaide a Darwin